In [1]:
import numpy as np
import pandas as pd

In [2]:
my_dict={'blood pressure':['T','F','F','F','T','T','F','T','T','F','F','T','T','T'],
         'CL_level':['N','N','C','H','C','H','H','N','C','N','C','H','N','H'],
         'Cigarette':['F','T','F','T','T','T','F','T','F','F','T','F','T','F'],
         'Weight':['OW','N','OW','OW','F','N','F','N','F','OW','N','OW','OW','F'],
         'cond':['T','F','T','T','T','T','F','T','T','F','T','F','T','F']}

In [3]:
df=pd.DataFrame(my_dict)
df

,blood pressure,CL_level,Cigarette,Weight,cond
0,T,N,F,OW,T
1,F,N,T,N,F
2,F,C,F,OW,T
3,F,H,T,OW,T
4,T,C,T,F,T
5,T,H,T,N,T
6,F,H,F,F,F
7,T,N,T,N,T
8,T,C,F,F,T
9,F,N,F,OW,F


In [4]:
def entropy(target_col):
    """
    Calculate the entropy of a dataset.
    The only parameter of this function is the target_col parameter which specifies the target column
    """
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy


In [5]:
def InfoGain(data,split_attribute_name,target_name="cond"):
    """
    Calculate the information gain of a dataset. This function takes three parameters:
    1. data = The dataset for whose feature the IG should be calculated
    2. split_attribute_name = the name of the feature for which the information gain should be calculated
    3. target_name = the name of the target feature. The default for this example is "class"
    """    
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [41]:
print(InfoGain(df,'blood pressure','cond'))
print(InfoGain(df,'CL_level','cond'))
print(InfoGain(df,'Cigarette','cond'))
print(InfoGain(df,'Weight','cond'))


0.04812703040826927
0.2467498197744391
0.15183550136234136
0.029222565658954647


In [44]:
df[df['CL_level']=='C'].drop('CL_level',axis=1)

,blood pressure,Cigarette,Weight,cond
2,F,F,OW,T
4,T,T,F,T
8,T,F,F,T
10,F,T,N,T


In [46]:
df_new=df[df['CL_level']=='H'].drop('CL_level',axis=1)
df_new

,blood pressure,Cigarette,Weight,cond
3,F,T,OW,T
5,T,T,N,T
6,F,F,F,F
11,T,F,OW,F
13,T,F,F,F


In [47]:
print(InfoGain(df_new,'blood pressure','cond'))
print(InfoGain(df_new,'Cigarette','cond'))
print(InfoGain(df_new,'Weight','cond'))

0.01997309402197489
0.9709505944546686
0.5709505944546686


In [50]:
df_new[df_new['Cigarette']=='T'].drop('Cigarette',axis=1)

,blood pressure,Weight,cond
3,F,OW,T
5,T,N,T


In [51]:
df_new[df_new['Cigarette']=='F'].drop('Cigarette',axis=1)

,blood pressure,Weight,cond
6,F,F,F
11,T,OW,F
13,T,F,F


In [52]:
df_new1=df[df['CL_level']=='N'].drop(['CL_level','Cigarette'],axis=1)
df_new1

,blood pressure,Weight,cond
0,T,OW,T
1,F,N,F
7,T,N,T
9,F,OW,F
12,T,OW,T


In [53]:
print(InfoGain(df_new1,'blood pressure','cond'))
print(InfoGain(df_new1,'Weight','cond'))

0.9709505944546686
0.01997309402197489


In [55]:
df_new1[df_new1['blood pressure']=='F'].drop('blood pressure',axis=1)

,Weight,cond
1,N,F
9,OW,F


In [56]:
from sklearn.tree import DecisionTreeClassifier

In [57]:
model = DecisionTreeClassifier(criterion='entropy')

In [82]:
X=df.drop("cond",axis=1)
y=df["cond"]

In [83]:
 from sklearn import preprocessing

In [84]:
le=preprocessing.LabelEncoder()

In [85]:
y.values

array(['T', 'F', 'T', 'T', 'T', 'T', 'F', 'T', 'T', 'F', 'T', 'F', 'T',
       'F'], dtype=object)

In [96]:
X_encode=np.array([le.fit_transform(col) for col in X.values.T]).T
#y_encode=le.fit_transform(y)
#y_encode
X_encode

array([[1, 2, 0, 2],
       [0, 2, 1, 1],
       [0, 0, 0, 2],
       [0, 1, 1, 2],
       [1, 0, 1, 0],
       [1, 1, 1, 1],
       [0, 1, 0, 0],
       [1, 2, 1, 1],
       [1, 0, 0, 0],
       [0, 2, 0, 2],
       [0, 0, 1, 1],
       [1, 1, 0, 2],
       [1, 2, 1, 2],
       [1, 1, 0, 0]])

In [92]:
model.fit(X_encode,y)

DecisionTreeClassifier(criterion='entropy')

In [93]:
base_pred = model.predict(X_encode)
base_pred

array(['T', 'F', 'T', 'T', 'T', 'T', 'F', 'T', 'T', 'F', 'T', 'F', 'T',
       'F'], dtype=object)

In [89]:
from sklearn.tree import plot_tree

In [7]:
df['Weight'][df['Weight']=='OW'].shape[0]

6

In [8]:
df['Weight'][df['Weight']=='F'].shape[0]

4

In [9]:
df['Weight'][df['Weight']=='N'].shape[0]

4

In [10]:
df['Weight'][df['Weight']=='OW'].shape[0]/14*entropy(df['cond'][df['Weight']=='OW'])

0.39355535745192405

In [11]:
df['Weight'][df['Weight']=='F'].shape[0]/14*entropy(df['cond'][df['Weight']=='F'])

0.2857142857142857

In [12]:
df['Weight'][df['Weight']=='N'].shape[0]/14*entropy(df['cond'][df['Weight']=='N'])

0.23179374984546652

In [13]:
entropy(df['cond'])-(df['Weight'][df['Weight']=='OW'].shape[0]/14*entropy(df['cond'][df['Weight']=='OW'])+df['Weight'][df['Weight']=='F'].shape[0]/14*entropy(df['cond'][df['Weight']=='F'])+df['Weight'][df['Weight']=='N'].shape[0]/14*entropy(df['cond'][df['Weight']=='N']))

0.029222565658954647

In [14]:
df_test=df.drop('cond',axis=1)
df_test

,blood pressure,CL_level,Cigarette,Weight
0,T,N,F,OW
1,F,N,T,N
2,F,C,F,OW
3,F,H,T,OW
4,T,C,T,F
5,T,H,T,N
6,F,H,F,F
7,T,N,T,N
8,T,C,F,F
9,F,N,F,OW


In [15]:
for i in df_test.columns:
  print(i)

blood pressure
CL_level
Cigarette
Weight


In [16]:

IG=np.zeros(4)
for i,col in enumerate(df_test.columns):
  IG[i]=InfoGain(df,col)
  print(IG[i])
print(np.max(IG))
sel_col=df_test.columns[np.argmax(IG)]
print(df_test.columns[np.argmax(IG)])


0.04812703040826927
0.2467498197744391
0.15183550136234136
0.029222565658954647
0.2467498197744391
CL_level


In [17]:
df_test

,blood pressure,CL_level,Cigarette,Weight
0,T,N,F,OW
1,F,N,T,N
2,F,C,F,OW
3,F,H,T,OW
4,T,C,T,F
5,T,H,T,N
6,F,H,F,F
7,T,N,T,N
8,T,C,F,F
9,F,N,F,OW


In [18]:
df[sel_col].unique()

array(['N', 'C', 'H'], dtype=object)

In [19]:
np.unique(df[sel_col],return_counts=True)[1]

array([4, 5, 5])

In [20]:
target_attribute_name='cond'

In [21]:
np.argmax(np.unique(df[target_attribute_name],return_counts=True)[1])

1

In [22]:
 np.unique(df[target_attribute_name])[np.argmax(np.unique(df[target_attribute_name],return_counts=True)[1])]

'T'

In [23]:
df[target_attribute_name].mode()[0]

'T'

In [24]:
InfoGain(df[df[sel_col]=='C'],'Weight')

0.0

In [25]:
len(df[sel_col].unique())

3

In [26]:
IG=np.zeros((len(df[sel_col].unique()),3))
df_test=df_test.drop(sel_col,axis=1)
for i,item in enumerate(df[sel_col].unique()):
  #df1=df[df[sel_col]==item]
  IG[i]=np.array([InfoGain(df[df[sel_col]==item],col) for col in df_test.columns])
IG

array([[0.97095059, 0.01997309, 0.01997309],
       [0.        , 0.        , 0.        ],
       [0.01997309, 0.97095059, 0.57095059]])

In [27]:
IG[1]

array([0., 0., 0.])

In [28]:
sel_leaf={}
for i in range(IG.shape[0]):
  if np.max(IG[i])==0:
    print("{}->{} : {}".format(sel_col,df[sel_col].unique()[i],df[df[sel_col]=='C']['cond'].unique()))
  else:
    sel_leaf[i]=df_test.columns[np.argmax(IG[i])]
sel_leaf

CL_level->C : ['T']


{0: 'blood pressure', 2: 'Cigarette'}

In [29]:
len(df.drop(target_attribute_name,axis=1).columns)

4

In [30]:
df['cond'].nunique()

2

In [36]:
def ID3(data,originaldata,features,target_attribute_name="cond",parent_node_class = None,J=[],current_depth=0,max_depth=1,j=None,depth_mode=False):
    #Define the stopping criteria --> If one of this is satisfied, we want to return a leaf node#
    
    #If all target_values have the same value, return this value
    if len(np.unique(data[target_attribute_name])) <= 1:
      #print("current depth : {}".format(current_depth))
      return np.unique(data[target_attribute_name])[0]
    
    #If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data)==0:
      #print("current depth : {}".format(current_depth))
      return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    #If the feature space is empty, return the mode target feature value of the direct parent node --> Note that
    #the direct parent node is that node which has called the current run of the ID3 algorithm and hence
    #the mode target feature value is stored in the parent_node_class variable.
    
    elif len(features) ==0 : 
      #print("current depth : {}".format(current_depth))
      return parent_node_class
    
    #If none of the above holds true, grow the tree!
    elif (depth_mode and current_depth==max_depth):
      return data[target_attribute_name].mode()[0]
    else:
        #Set the default value for this node --> The mode target feature value of the current node
        #parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        print(data)
        parent_node_class = data[target_attribute_name].mode()[0]
        print("P-node : {}".format(parent_node_class))
        #Select the feature which best splits the dataset
        best_feature  = features[np.argmax([InfoGain(data,feature,target_attribute_name) for feature in features])] #Return the information gain values for the features in the dataset 
        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        
        tree = {best_feature:{}}
        #print("tree : {}".format(tree))
        
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]
        
        print("This is best-feature : {}".format(best_feature))

        #Grow a branch under the root node for each possible value of the root node feature
        for j,value in enumerate(np.unique(data[best_feature])):
          J.append(j)
          if j==0:
            current_depth+=1
          #print("This is value : {}".format(value))
          #print("j : {}".format(j))
          #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
          sub_data = data[data[best_feature] == value]
            
          #Call the ID3 algorithm for each of those sub_datasets with the new parameters -->  recursive part!
          #print(i)  
          
          subtree = ID3(sub_data,data,features,target_attribute_name,parent_node_class,J,current_depth,max_depth,j)
          
          #print("This is subtree : {}".format(subtree))
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
          tree[best_feature][value] = subtree
          print("this is tree : {}".format(tree))
        #print(J) 
        return(tree) 
          
                
"""
Train the tree, Print the tree and predict the accuracy
"""
tree = ID3(df,df,df.columns[:-1])
print(tree)


   blood pressure CL_level Cigarette Weight cond
0               T        N         F     OW    T
1               F        N         T      N    F
2               F        C         F     OW    T
3               F        H         T     OW    T
4               T        C         T      F    T
5               T        H         T      N    T
6               F        H         F      F    F
7               T        N         T      N    T
8               T        C         F      F    T
9               F        N         F     OW    F
10              F        C         T      N    T
11              T        H         F     OW    F
12              T        N         T     OW    T
13              T        H         F      F    F
P-node : T
This is best-feature : CL_level
this is tree : {'CL_level': {'C': 'T'}}
   blood pressure CL_level Cigarette Weight cond
3               F        H         T     OW    T
5               T        H         T      N    T
6               F        H         

In [98]:
df.columns[:-1]

Index(['blood pressure', 'CL_level', 'Cigarette', 'Weight'], dtype='object')

In [99]:
for value in np.unique(df['CL_level']):
  #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
  sub_data = df[df['CL_level'] == value]
  print(sub_data)

   blood pressure CL_level Cigarette Weight cond
2               F        C         F     OW    T
4               T        C         T      F    T
8               T        C         F      F    T
10              F        C         T      N    T
   blood pressure CL_level Cigarette Weight cond
3               F        H         T     OW    T
5               T        H         T      N    T
6               F        H         F      F    F
11              T        H         F     OW    F
13              T        H         F      F    F
   blood pressure CL_level Cigarette Weight cond
0               T        N         F     OW    T
1               F        N         T      N    F
7               T        N         T      N    T
9               F        N         F     OW    F
12              T        N         T     OW    T


In [100]:
my_dict_test={'blood pressure':['T','T','T','T','F'],
              'CL_level':['N','H','H','N','N'],
              'Cigarette':['T','T','F','F','T'],
              'Weight':['F','F','N','N','OW'],
              'cond':['T','T','F','F','T']}

In [101]:
df_test=pd.DataFrame(my_dict_test)
df_test

,blood pressure,CL_level,Cigarette,Weight,cond
0,T,N,T,F,T
1,T,H,T,F,T
2,T,H,F,N,F
3,T,N,F,N,F
4,F,N,T,OW,T


In [109]:
def predict(query,tree,default = 1):
    #1.
    for key in list(query.keys()):
        if key in list(tree.keys()):
            #2.
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            #3.
            result = tree[key][query[key]]
            #4.
            if isinstance(result,dict):
                return predict(query,result)

            else:
                return result

In [107]:
def test(data,tree):
    #Create new query instances by simply removing the target feature column from the original dataset and 
    #convert it to a dictionary
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 

    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print(predicted)
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["cond"])/len(data))*100,'%')


In [108]:
test(df_test,tree)

  predicted
0         T
1         T
2         F
3         T
4         F
The prediction accuracy is:  60.0 %
